In [11]:
%reload_ext autoreload

"""
This script runs classifier training over the entire training data and then
output predictions over the interactome.

Usage:
  predict_ppis.py [--interpro] [--pfam] [--mf] [--cc] [--bp]
                  [--use_cache] [--retrain] [--induce] [--verbose]
                  [--model=M] [--n_jobs=J] [--n_splits=S] [--n_iterations=I]
                  [--input=FILE] [--output=FILE] [--directory=DIR]
  predict_ppis.py -h | --help

Options:
  -h --help     Show this screen.
  --interpro    Use interpro domains in features.
  --pfam        Use Pfam domains in features.
  --mf          Use Molecular Function Gene Ontology in features.
  --cc          Use Cellular Compartment Gene Ontology in features.
  --bp          Use Biological Process Gene Ontology in features.
  --induce      Use ULCA inducer over Gene Ontology.
  --verbose     Print intermediate output for debugging.
  --binary      Use binary feature encoding instead of ternary.
  --use_cache   Use cached features if available.
  --retrain     Re-train classifier instead of loading previous version. If
                using a previous version, you must use the same selection of
                features along with the same induce setting.
  --model=M         A binary classifier from Scikit-Learn implementing fit,
                    predict and predict_proba [default: LogisticRegression].
                    Ignored if using 'retrain'.
  --n_jobs=J        Number of processes to run in parallel [default: 1]
  --n_splits=S      Number of cross-validation splits used during randomized
                    grid search [default: 5]
  --n_iterations=I  Number of randomized grid search iterations [default: 60]
  --input=FILE      Uniprot edge-list, with a path directory that absolute or
                    relative to this script. Entries must be tab separated with
                    header columns 'source' and 'target'. [default: 'default']
  --output=FILE     Output file name [default: predictions.tsv]
  --directory=DIR   Absolute or relative output directory [default: ./results/]
"""

import os
import json
import logging
import numpy as np
from datetime import datetime

from pyppi.base import parse_args, su_make_dir
from pyppi.base import P1, P2, G1, G2
from pyppi.data import load_network_from_path, load_ptm_labels
from pyppi.data import full_training_network_path, generic_io
from pyppi.data import interactome_network_path, classifier_path

from pyppi.models import make_classifier, get_parameter_distribution_for_model

from pyppi.data_mining.features import AnnotationExtractor
from pyppi.data_mining.uniprot import UniProt, get_active_instance
from pyppi.data_mining.tools import xy_from_interaction_frame
from pyppi.data_mining.generic import edgelist_func, generic_to_dataframe
from pyppi.data_mining.tools import map_network_accessions

from sklearn.externals import joblib
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multiclass import OneVsRestClassifier

logging.captureWarnings(False)
logging.basicConfig(
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%m-%d-%Y %I:%M:%S',
    level=logging.DEBUG,
)
logger = logging.getLogger(__name__)

args = dict(
    n_jobs=3,
    n_splits=2,
    rcv_iter=10,
    induce=True,
    verbose=True,
    selection = [
        UniProt.data_types().GO_MF.value,
        UniProt.data_types().GO_BP.value,
        UniProt.data_types().GO_CC.value,
        UniProt.data_types().INTERPRO.value,
        UniProt.data_types().PFAM.value
    ],
    model='LogisticRegression',
    use_cache=True,
    output='predictions.tsv',
    input='test_network.tsv',
    direc='./results/',
    retrain=True,
    use_binary=False
)
n_jobs = args['n_jobs']
n_splits = args['n_splits']
rcv_iter = args['rcv_iter']
induce = args['induce']
verbose = args['verbose']
selection = args['selection']
model = args['model']
use_feature_cache = args['use_cache']
out_file = args['output']
input_file = args['input']
direc = args['direc']
retrain = args['retrain']
use_binary = args['use_binary']

In [2]:
# Set up the folder for each experiment run named after the current time
# -------------------------------------------------------------------- #
folder = datetime.now().strftime("pred_%y-%m-%d_%H-%M-%S")
direc = "{}/{}/".format(direc, folder)
su_make_dir(direc)
json.dump(args, fp=open("{}/settings.json".format(direc), 'w'), indent=4,
          sort_keys=True)
out_file = open("{}/{}".format(direc, out_file), "w")

In [3]:
# Load features from feature cache, or create an empty annotation extrator
# ------------------------------------------------------------------------ #
logger.info("Loading feature data...")
uniprot = get_active_instance(verbose=verbose)
data_types = UniProt.data_types()
labels = load_ptm_labels()
annotation_ex = AnnotationExtractor(
    induce=induce,
    selection=selection,
    n_jobs=n_jobs,
    verbose=verbose,
    cache=use_feature_cache
)

[11-17-2017 05:58:39] INFO: Loading feature data...


First time loading on UniProt instance. Make take a few moments


In [9]:
# Get the input edge-list ready
# -------------------------------------------------------------------- #
if input_file == 'default':
    logger.info("Loading interactome data...")
    testing = load_network_from_path(interactome_network_path)
else:
    logger.info("Loading custom ppi data...")
    testing = generic_to_dataframe(
        f_input=generic_io(input_file),
        parsing_func=edgelist_func,
        drop_nan=True,
        allow_self_edges=True,
        allow_duplicates=True
    )
    sources = set(p for p in testing.source.values)
    targets = set(p for p in testing.target.values)
    accessions = list(sources | targets)
    accession_mapping = uniprot.batch_map(accessions)
    testing = map_network_accessions(
        interactions=testing, accession_map=accession_mapping,
        drop_nan=True, allow_self_edges=True,
        allow_duplicates=False, min_counts=None, merge=False
    )

In [15]:
# Get the features into X, and multilabel y indicator format
# -------------------------------------------------------------------- #
logger.info("Preparing training and testing data...")
training = load_network_from_path(full_training_network_path)
X_train_ppis, y_train = xy_from_interaction_frame(training)
X_test_ppis, _ = xy_from_interaction_frame(testing)

if not use_feature_cache:
    logger.info("Computing feature cache.")
    annotation_ex.fit(X_train_ppis + X_test_ppis)
    X_train = annotation_ex.transform(X_train_ppis)
    X_test = annotation_ex.transform(X_test_ppis)
else:
    try:
        X_train = annotation_ex.transform(X_train_ppis)
        X_test = annotation_ex.transform(X_test_ppis)
    except ValueError:
        logger.info("Found new PPIs. Re-computing feature cache.")
        annotation_ex.fit(X_train_ppis + X_test_ppis)
        X_train = annotation_ex.transform(X_train_ppis)
        X_test = annotation_ex.transform(X_test_ppis)

# Get all annotations used during training
# -------------------------------------------------------------------- #
training_go = set([
    g.strip() for gs in X_train
    for g in x.split(',')
    if 'go' in g.strip().lower()
])
training_pfam = set([
    g.strip() for gs in X_train
    for g in x.split(',')
    if 'pf' in g.strip().lower()
])
training_ipr = set([
    g.strip() for gs in X_train
    for g in x.split(',')
    if 'ipr' in g.strip().lower()
])

mlb = MultiLabelBinarizer(classes=sorted(labels))
mlb.fit(y_train)
y_train = mlb.transform(y_train)

vectorizer = CountVectorizer(binary=True if use_binary else False)
X_train = vectorizer.fit_transform(X_train)

Acquiring features for PPIs...
Acquiring data for accession 1/4029..
Acquiring data for accession 2/4029..
Acquiring data for accession 3/4029..
Acquiring data for accession 4/4029..
Acquiring data for accession 5/4029..
Acquiring data for accession 6/4029..
Acquiring data for accession 7/4029..
Acquiring data for accession 8/4029..
Acquiring data for accession 9/4029..
Acquiring data for accession 10/4029..
Acquiring data for accession 11/4029..
Acquiring data for accession 12/4029..
Acquiring data for accession 13/4029..
Acquiring data for accession 14/4029..
Acquiring data for accession 15/4029..
Acquiring data for accession 16/4029..
Acquiring data for accession 17/4029..
Acquiring data for accession 18/4029..
Acquiring data for accession 19/4029..
Acquiring data for accession 20/4029..
Acquiring data for accession 21/4029..
Acquiring data for accession 22/4029..
Acquiring data for accession 23/4029..
Acquiring data for accession 24/4029..
Acquiring data for accession 25/4029..
Acq

Acquiring data for accession 207/4029..
Acquiring data for accession 208/4029..
Acquiring data for accession 209/4029..
Acquiring data for accession 210/4029..
Acquiring data for accession 211/4029..
Acquiring data for accession 212/4029..
Acquiring data for accession 213/4029..
Acquiring data for accession 214/4029..
Acquiring data for accession 215/4029..
Acquiring data for accession 216/4029..
Acquiring data for accession 217/4029..
Acquiring data for accession 218/4029..
Acquiring data for accession 219/4029..
Acquiring data for accession 220/4029..
Acquiring data for accession 221/4029..
Acquiring data for accession 222/4029..
Acquiring data for accession 223/4029..
Acquiring data for accession 224/4029..
Acquiring data for accession 225/4029..
Acquiring data for accession 226/4029..
Acquiring data for accession 227/4029..
Acquiring data for accession 228/4029..
Acquiring data for accession 229/4029..
Acquiring data for accession 230/4029..
Acquiring data for accession 231/4029..


Acquiring data for accession 411/4029..
Acquiring data for accession 412/4029..
Acquiring data for accession 413/4029..
Acquiring data for accession 414/4029..
Acquiring data for accession 415/4029..
Acquiring data for accession 416/4029..
Acquiring data for accession 417/4029..
Acquiring data for accession 418/4029..
Acquiring data for accession 419/4029..
Acquiring data for accession 420/4029..
Acquiring data for accession 421/4029..
Acquiring data for accession 422/4029..
Acquiring data for accession 423/4029..
Acquiring data for accession 424/4029..
Acquiring data for accession 425/4029..
Acquiring data for accession 426/4029..
Acquiring data for accession 427/4029..
Acquiring data for accession 428/4029..
Acquiring data for accession 429/4029..
Acquiring data for accession 430/4029..
Acquiring data for accession 431/4029..
Acquiring data for accession 432/4029..
Acquiring data for accession 433/4029..
Acquiring data for accession 434/4029..
Acquiring data for accession 435/4029..


Acquiring data for accession 615/4029..
Acquiring data for accession 616/4029..
Acquiring data for accession 617/4029..
Acquiring data for accession 618/4029..
Acquiring data for accession 619/4029..
Acquiring data for accession 620/4029..
Acquiring data for accession 621/4029..
Acquiring data for accession 622/4029..
Acquiring data for accession 623/4029..
Acquiring data for accession 624/4029..
Acquiring data for accession 625/4029..
Acquiring data for accession 626/4029..
Acquiring data for accession 627/4029..
Acquiring data for accession 628/4029..
Acquiring data for accession 629/4029..
Acquiring data for accession 630/4029..
Acquiring data for accession 631/4029..
Acquiring data for accession 632/4029..
Acquiring data for accession 633/4029..
Acquiring data for accession 634/4029..
Acquiring data for accession 635/4029..
Acquiring data for accession 636/4029..
Acquiring data for accession 637/4029..
Acquiring data for accession 638/4029..
Acquiring data for accession 639/4029..


Acquiring data for accession 819/4029..
Acquiring data for accession 820/4029..
Acquiring data for accession 821/4029..
Acquiring data for accession 822/4029..
Acquiring data for accession 823/4029..
Acquiring data for accession 824/4029..
Acquiring data for accession 825/4029..
Acquiring data for accession 826/4029..
Acquiring data for accession 827/4029..
Acquiring data for accession 828/4029..
Acquiring data for accession 829/4029..
Acquiring data for accession 830/4029..
Acquiring data for accession 831/4029..
Acquiring data for accession 832/4029..
Acquiring data for accession 833/4029..
Acquiring data for accession 834/4029..
Acquiring data for accession 835/4029..
Acquiring data for accession 836/4029..
Acquiring data for accession 837/4029..
Acquiring data for accession 838/4029..
Acquiring data for accession 839/4029..
Acquiring data for accession 840/4029..
Acquiring data for accession 841/4029..
Acquiring data for accession 842/4029..
Acquiring data for accession 843/4029..


...

Computing selected features for PPIs...


[Parallel(n_jobs=1)]: Done   0 out of   0 | elapsed:    0.0s finished


Updating instance attributes...


[Parallel(n_jobs=1)]: Done   0 out of   0 | elapsed:    0.0s finished


KeyError: 'accession'

In [21]:
annotation_ex._ppi_df

Index([], dtype='object')

In [ ]:
# Make the estimators and BR classifier
# -------------------------------------------------------------------- #
if retrain or not os.path.isfile(classifier_path):
    logger.info("Making classifier...")
    params = get_parameter_distribution_for_model(model)
    random_cv = RandomizedSearchCV(
        cv=n_splits,
        n_iter=rcv_iter,
        n_jobs=n_jobs,
        refit=True, 
        random_state=0,
        scoring='f1', 
        error_score=0,
        param_distributions=params,
        estimator=make_classifier(model)
    )
    clf = OneVsRestClassifier(estimator=random_cv, n_jobs=1)

    # Fit the complete training data and make predictions.
    logging.info("Fitting data...")
    clf.fit(X_train, y_train)
    joblib.dump(clf, classifier_path)

In [ ]:
# Loads a previously (or recently trained) classifier from disk
# and then performs the predictions on the new dataset.
# -------------------------------------------------------------------- #
logging.info("Making predictions...")
clf = joblib.load(classifier_path)
predictions = clf.predict_proba(X_test)

In [ ]:
# Write the predictions to a tsv file
# -------------------------------------------------------------------- #
logging.info("Writing results to file...")
header = "{p1}\t{p2}\t{g1}\t{g2}\t{classes}\tsum\n".format(
    p1=P1, p2=P2, g1=G1, g2=G2, classes='\t'.join(sorted(mlb.classes_))
)
out_file.write(header)
acc = annotation_ex.accession_vocabulary[UniProt.accession_column()]
genes = annotation_ex.accession_vocabulary[UniProt.data_types().GENE.value]
accession_gene_map = {a: g for (a, g) in zip(acc, genes)}
for (s, t), p_vec in zip(X_test_ppis, predictions):
    p_vec = [p for _, p in sorted(zip(mlb.classes_, p_vec))]
    g1 = accession_gene_map.get(s, ['-'])[0] or '-'
    g2 = accession_gene_map.get(t, ['-'])[0] or '-'

    # Compute the usability of each of the annotation sets
    annots = annotation_ex.transform([s, t])
    go = set([
        g.strip() for gs in annots
        for g in x.split(',')
        if 'go' in g.strip().lower()
    ])
    pf = set([
        g.strip() for gs in annots
        for g in x.split(',')
        if 'pf' in g.strip().lower()
    ])
    ipr = set([
        g.strip() for gs in annots
        for g in x.split(',')
        if 'ipr' in g.strip().lower()
    ])
    usability_go = (go & training_go) / (go | training_go)
    usability_pf = (pf & training_pfam) / (pf | training_pfam)
    usability_ipr = (ipr & training_ipr) / (ipr | training_ipr)

    sum_pr = sum(p_vec)
    line = "{s}\t{t}\t{g1}\t{g2}\t{classes}\t{sum_pr}\t{usability_go}" \
           "\t{usability_pf}\t{usability_ipr}\n".format(
            s=s, t=t, g1=g1, g2=g2, sum_pr=sum_pr,
            classes='\t'.join(['%.4f' % p for p in p_vec]),
            usability_go=usability_go,
            usability_pf=usability_pf,
            usability_ipr=usability_ipr)
    out_file.write(line)
out_file.close()